# MongoDB Aggregation Examples

- Ensure you have pymongo installed before running cells in this notebook

In [1]:
import pymongo
from bson.json_util import dumps
import pprint

#  -->  Update the URI with your username and password <--

uri = "mongodb://localhost:27017/"
client = pymongo.MongoClient(uri)
mflixdb = client.mflix
demodb = client.demodb

## About Aggregates in PyMongo

- Aggregation uses _pipelines_.
- A **pipeline** is a sequence of stages through which documents proceed.
- Some of the different stages that can be used are:
  - match
  - project
  - sort
  - limit
  - unwind
  - group
  - lookup

### $match

In [2]:
c = mflixdb.movies.aggregate([
    {"$match": {"year": {"$lte": 1920}}},
])

print(dumps(c, indent=4))

[
    {
        "_id": {
            "$oid": "573a1390f29313caabcd42e8"
        },
        "plot": "A group of bandits stage a brazen train hold-up, only to find a determined posse hot on their heels.",
        "genres": [
            "Short",
            "Western"
        ],
        "runtime": 11,
        "cast": [
            "A.C. Abadie",
            "Gilbert M. 'Broncho Billy' Anderson",
            "George Barnes",
            "Justus D. Barnes"
        ],
        "poster": "https://m.media-amazon.com/images/M/MV5BMTU3NjE5NzYtYTYyNS00MDVmLWIwYjgtMmYwYWIxZDYyNzU2XkEyXkFqcGdeQXVyNzQzNzQxNzI@._V1_SY1000_SX677_AL_.jpg",
        "title": "The Great Train Robbery",
        "fullplot": "Among the earliest existing films in American cinema - notable as the first film that presented a narrative story to tell - it depicts a group of cowboy outlaws who hold up a train and rob the passengers. They are then pursued by a Sheriff's posse. Several scenes have color included - all hand tinted.",


### match and project

In [3]:
c = mflixdb.movies.aggregate([
    {"$match": {"year": {"$lte": 1920}}},
    {"$project": {"_id":0, "title": 1, "cast": 1}},
])

print(dumps(c, indent=4))

[
    {
        "cast": [
            "A.C. Abadie",
            "Gilbert M. 'Broncho Billy' Anderson",
            "George Barnes",
            "Justus D. Barnes"
        ],
        "title": "The Great Train Robbery"
    },
    {
        "cast": [
            "Frank Powell",
            "Grace Henderson",
            "James Kirkwood",
            "Linda Arvidson"
        ],
        "title": "A Corner in Wheat"
    },
    {
        "cast": [
            "Winsor McCay"
        ],
        "title": "Winsor McCay, the Famous Cartoonist of the N.Y. Herald and His Moving Comics"
    },
    {
        "cast": [
            "Jane Gail",
            "Ethel Grandin",
            "William H. Turner",
            "Matt Moore"
        ],
        "title": "Traffic in Souls"
    },
    {
        "cast": [
            "Winsor McCay",
            "George McManus",
            "Roy L. McCardell"
        ],
        "title": "Gertie the Dinosaur"
    },
    {
        "cast": [
            "Stanley Hunt",
 

### match project limit and sort

In [4]:
c = mflixdb.movies.aggregate([
    {"$match": {"year": {"$lte": 1920}}},
    {"$sort": {"title": 1}},
    {"$limit": 5},
    {"$project": {"_id":0, "title": 1, "cast": 1}},
])

print(dumps(c, indent=4))

[
    {
        "cast": [
            "Frank Powell",
            "Grace Henderson",
            "James Kirkwood",
            "Linda Arvidson"
        ],
        "title": "A Corner in Wheat"
    },
    {
        "cast": [
            "Howard C. Hickman",
            "Enid Markey",
            "Lola May",
            "Kate Bruce"
        ],
        "title": "Civilization"
    },
    {
        "cast": [
            "Harold Lloyd",
            "Mildred Davis",
            "'Snub' Pollard",
            "Peggy Cartwright"
        ],
        "title": "From Hand to Mouth"
    },
    {
        "cast": [
            "Winsor McCay",
            "George McManus",
            "Roy L. McCardell"
        ],
        "title": "Gertie the Dinosaur"
    },
    {
        "cast": [
            "Harold Lloyd",
            "Roy Brooks",
            "Mildred Davis",
            "Wallace Howe"
        ],
        "title": "High and Dizzy"
    }
]


### Unwind

In [5]:
c = mflixdb.movies.aggregate([
    {"$match": {"year": {"$lte": 1920}}},
    {"$sort": {"imdb.rating": -1}},
    {"$limit": 5},
    {"$unwind": "$cast"},
    {"$project": {"_id":0, "title": 1, "cast": 1, "rating": "$imdb.rating"}},
])

print(dumps(c, indent=4))

[
    {
        "cast": "Buster Keaton",
        "title": "One Week",
        "rating": 8.3
    },
    {
        "cast": "Sybil Seely",
        "title": "One Week",
        "rating": 8.3
    },
    {
        "cast": "Pearl White",
        "title": "The Perils of Pauline",
        "rating": 7.6
    },
    {
        "cast": "Crane Wilbur",
        "title": "The Perils of Pauline",
        "rating": 7.6
    },
    {
        "cast": "Paul Panzer",
        "title": "The Perils of Pauline",
        "rating": 7.6
    },
    {
        "cast": "Edward Jos\u00e8",
        "title": "The Perils of Pauline",
        "rating": 7.6
    },
    {
        "cast": "A.C. Abadie",
        "title": "The Great Train Robbery",
        "rating": 7.4
    },
    {
        "cast": "Gilbert M. 'Broncho Billy' Anderson",
        "title": "The Great Train Robbery",
        "rating": 7.4
    },
    {
        "cast": "George Barnes",
        "title": "The Great Train Robbery",
        "rating": 7.4
    },
    {
      

## Grouping

In [6]:
# What is the average IMDB rating of all movies by year? sort the data by year.

c = mflixdb.movies.aggregate([
    {"$group": {"_id": {"release year": "$year"}, "Avg Rating": {"$avg": "$imdb.rating"}}},
    {"$sort" : {"_id": 1}}
    
])
print(dumps(c, indent = 2))

[
  {
    "_id": {
      "release year": 1896
    },
    "Avg Rating": 5.9
  },
  {
    "_id": {
      "release year": 1903
    },
    "Avg Rating": 7.4
  },
  {
    "_id": {
      "release year": 1909
    },
    "Avg Rating": 6.6
  },
  {
    "_id": {
      "release year": 1911
    },
    "Avg Rating": 7.3
  },
  {
    "_id": {
      "release year": 1913
    },
    "Avg Rating": 6.0
  },
  {
    "_id": {
      "release year": 1914
    },
    "Avg Rating": 6.8999999999999995
  },
  {
    "_id": {
      "release year": 1915
    },
    "Avg Rating": 6.6
  },
  {
    "_id": {
      "release year": 1916
    },
    "Avg Rating": 6.1
  },
  {
    "_id": {
      "release year": 1917
    },
    "Avg Rating": 6.9
  },
  {
    "_id": {
      "release year": 1918
    },
    "Avg Rating": 6.6
  },
  {
    "_id": {
      "release year": 1919
    },
    "Avg Rating": 7.0
  },
  {
    "_id": {
      "release year": 1920
    },
    "Avg Rating": 6.9750000000000005
  },
  {
    "_id": {
      "release 

In [ ]:
# What is the average IMDB rating of all movies by year? sort the data by avg rating in decreasing order.

c = mflixdb.movies.aggregate([
    {"$group": {"_id": {"release year": "$year"}, "Avg Rating": {"$avg": "$imdb.rating"}}},
    {"$sort" : {"Avg Rating": -1, "_id": 1}}
    
])
print(dumps(c, indent = 2))

## Lookup

In [7]:
data = demodb.customers.aggregate([
    {
        "$lookup": {
            "from": "orders",
            "localField": "custid",
            "foreignField": "custid",
            "as": "orders"
        }
    },
    {"$project": {"_id": 0, "address": 0}}
])
print(dumps(data, indent = 2))

[]


## Reformatting Queries

In [8]:
match = {"$match": {"year": {"$lte": 1920}}}
limit = {"$limit": 5}
project = {"$project": {"_id":0, "title": 1, "cast": 1, "rating": "$imdb.rating"}}

agg = mflixdb.movies.aggregate([match, limit, project])
print(dumps(agg, indent=2))

[
  {
    "cast": [
      "A.C. Abadie",
      "Gilbert M. 'Broncho Billy' Anderson",
      "George Barnes",
      "Justus D. Barnes"
    ],
    "title": "The Great Train Robbery",
    "rating": 7.4
  },
  {
    "cast": [
      "Frank Powell",
      "Grace Henderson",
      "James Kirkwood",
      "Linda Arvidson"
    ],
    "title": "A Corner in Wheat",
    "rating": 6.6
  },
  {
    "cast": [
      "Winsor McCay"
    ],
    "title": "Winsor McCay, the Famous Cartoonist of the N.Y. Herald and His Moving Comics",
    "rating": 7.3
  },
  {
    "cast": [
      "Jane Gail",
      "Ethel Grandin",
      "William H. Turner",
      "Matt Moore"
    ],
    "title": "Traffic in Souls",
    "rating": 6
  },
  {
    "cast": [
      "Winsor McCay",
      "George McManus",
      "Roy L. McCardell"
    ],
    "title": "Gertie the Dinosaur",
    "rating": 7.3
  }
]
